# <center>Big Data &ndash; Exercise 1 &ndash; Solutions</center>
## <center>Fall 2023 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](http://please.add.link.here) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [1]:
server='db'
user='postgres'
password='example'
database='postgres'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [2]:
%reload_ext sql
%sql $connection_string


In [3]:
%%sql
SELECT version();

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


version
"PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit"


### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [4]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/postgres
22 rows affected.


table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database we will be using.

In [5]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name IN ('artists', 'released_by', 'releases', 'tracks')
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

 * postgresql://postgres:***@db:5432/postgres
17 rows affected.


table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
released_by,release_id,integer,NO,1
released_by,artist_id,integer,NO,2
releases,release_id,integer,NO,1
releases,released,date,NO,2
releases,title,text,NO,3


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [6]:
%%sql
SELECT * FROM artists LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
1,The Persuader,Jesper Dahlbäck,None,None
2,Mr. James Barth & A.D.,Cari Lekebusch & Alexi Delano,None,None
3,Josh Wink,Joshua Winkelman,"After forming [l=Ovum Recordings] as an independent label in October 1994 with former partner [a=King Britt], Josh recorded the cult classic 'Liquid Summer'. He went on to release singles for a wide variety of revered European labels ranging from Belgium's [l=R & S Records] to England's [l=XL Recordings]. In 1995, Wink became one of the first DJ-producers to translate his hard work into mainstream success when he unleashed a string of classics including 'Don't Laugh'¸ 'I'm Ready' and 'Higher State of Consciousness' that topped charts worldwide. More recently he has had massive club hits such as 'How's Your Evening So Far' and 'Superfreak' but he has also gained a lot of attention trough his remixes for [a=FC Kahuna], [a=Paul Oakenfold], [a=Ladytron], [a=Clint Mansell], [a=Sting] and [a=Depeche Mode], among others.",http://www.joshwink.com/
4,Johannes Heil,Johannes Heil,"Electronic music producer, musician and live performer, born 3 February 1978 near the town of Bad Nauheim, Germany. Founder of [l=JH] and [l=Metatron Recordings].",http://johannes-heil.com/
5,Heiko Laux,Heiko Laux,German DJ and producer based in Berlin. He is the founder of [l=Kanzleramt].,http://www.heiko-laux.com


Naturally we could write similar queries to better understand each of the other tables.

#### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

### Solution

1. The dataset contains data from `discogs` and models musical artists, their physical releases, and the tracks on those releases. You can intuitively connect the tables with "artists release albums with tracks".

2. 
   - The number of concepts is limited, fixed, and well-defined.
   - The same is true for attributes of these concepts.
   - Attributes come from a well-defined domains with a fixed semantic (such as dates, e-mail addresses, ...).
   - Instances of concepts are in relation with each other, which may or may not be required to exist.
   - In short: we can define a schema, which the rest of the application (the website) relies on.


3. 
   - All relations are atomic so they are in 1NF.
   - `artists`, `releases` and `tracks` all have a singular ID primary key, so they at least 2NF. Upon inspection, there are no transitive dependencies, so they are in 3NF.
   - `released_by` does not contain non-key attributes, so it is in 3NF.


4. A very typical example of a query is finding all the releases by an artist's name. If we redundantly store the artist's ID and name in `releases`, the aforementioned query now only requires a `SELECT` instead of an expensive `JOIN`. As releases can be released by multiple artists, we could also extend this idea and break 1NF by storing a list of artist names instead. The tradeoff here is that every time we have to update an artist's name we have to find and update all releases by that artist. Considering that reads are very common and changes are rare, the option of denormalizing becomes relatively more attractive.

5. 
   - Insert anomaly: we could insert a release with a fake or incorrect artist name.
   - Update anomaly: if we update an artist's name but not all of the records in `releases`, we introduce an inconsistency into the dataset.

## Exercise 2: SQL warm-up
Let us begin with several SQL queries to ease us back into the language.

#### 1. Retrieve the artists with the name of 'Coldplay'.

In [7]:
%%sql
SELECT *
FROM artists
WHERE name = 'Coldplay';

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,realname,profile,url
29735,Coldplay,None,"Coldplay is an English rock band from London, England. They've been a band since January 16, 1998 when they lost a demotape competition on XFM in London. Philip Christopher Harvey is the band's manager. [b][u]Line-up:[/u][/b] Jonny Buckland (Jonathan Mark Buckland) - Guitar Will Champion (William Champion) - Drums Guy Berryman (Guy Rupert Berryman) - Bass Chris Martin (Christopher Anthony John Martin) - Vocals",http://coldplay.com/


#### 2. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

In [8]:
%%sql
SELECT DISTINCT releases.title
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
WHERE artists.name = 'Coldplay'
ORDER BY releases.title;

 * postgresql://postgres:***@db:5432/postgres
40 rows affected.


title
Acoustic
A Rush Of Blood To The Head
Boot Of Sound
Brothers & Sisters
Clocks
Clocks...
Clocks / Chime Trance Remixes
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Planet Rockers Remixes)


#### 3. How many tracks from 'Coldplay' have position '1'?

In [9]:
%%sql
SELECT COUNT(*)
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
JOIN tracks USING(release_id)
WHERE artists.name = 'Coldplay' AND tracks.position = '1';

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
32


#### 4. What is the average track duration?

In [10]:
%%sql
SELECT AVG(duration) FROM tracks;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


avg
324.59572773212363


#### 5. How many artists have released tracks longer than twice the average?

In [11]:
%%sql
WITH average_duration AS (SELECT AVG(CAST(duration AS FLOAT)) FROM tracks)

SELECT COUNT(DISTINCT(artists.artist_id))
FROM artists
JOIN released_by USING(artist_id)   
JOIN releases USING(release_id)
JOIN tracks USING(release_id)
WHERE tracks.duration > 2*(SELECT * FROM average_duration);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
18894


## Exercise 3: more SQL
We will now see more complex SQL queries.

#### 1. What is the title of the album from 'Coldplay' with the most amount of tracks?

In [12]:
%%sql
WITH tracks_per_album AS(SELECT releases.title AS album, COUNT(tracks.track_id) AS tracks
                         FROM artists
                         JOIN released_by USING(artist_id)   
                         JOIN releases USING(release_id)
                         JOIN tracks USING(release_id)
                         WHERE artists.name = 'Coldplay' 
                         GROUP BY releases.release_id, releases.title)

SELECT album, tracks
FROM tracks_per_album
WHERE tracks = (SELECT MAX(tracks)
                FROM tracks_per_album);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


album,tracks
X&Y (Special Dutch Edition),19


#### 2. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.5 since we are lookong at the <b>total</b> track duration of the album.

In [13]:
%%sql
WITH total_duration AS (SELECT release_id, SUM(duration) AS duration
                        FROM tracks
                        GROUP BY release_id), 
average AS (select AVG(duration)
            FROM total_duration)

SELECT COUNT(DISTINCT(artists.artist_id))
FROM artists
JOIN released_by USING(artist_id)   
JOIN releases USING(release_id)
JOIN total_duration USING(release_id)
WHERE duration > 2*(SELECT * FROM average);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
16426


#### 3. How many artists have both a release with a track longer than twice the average and one with total duration longer than twice the average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

In [14]:
%%sql
WITH total_duration AS (
    SELECT release_id, SUM(duration) AS duration
    FROM tracks
    GROUP BY release_id
),
average AS (
    SELECT AVG(duration) AS avg_duration
    FROM total_duration
),
average_duration AS (
    SELECT AVG(CAST(duration AS FLOAT)) AS avg_track_duration
    FROM tracks
),
artist_intersection AS (
    SELECT artist_id
    FROM artists
    JOIN released_by USING (artist_id)
    JOIN releases USING (release_id)
    JOIN total_duration USING (release_id)
    WHERE duration > 2*(SELECT avg_duration FROM average)
    INTERSECT
    SELECT artist_id
    FROM artists
    JOIN released_by USING (artist_id)
    JOIN releases USING (release_id)
    JOIN tracks USING (release_id)
    WHERE tracks.duration > 2*(SELECT avg_track_duration FROM average_duration)
)

SELECT COUNT(*)
FROM artist_intersection;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
8146


#### 4. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

In [15]:
%%sql
WITH big_guys AS (SELECT artist_id, name, COUNT(release_id) AS c
                  FROM artists NATURAL JOIN released_by NATURAL JOIN releases
                  GROUP BY artist_id, name
                  HAVING COUNT(release_id)>200)
SELECT name
FROM big_guys
WHERE artist_id NOT IN (SELECT artist_id
                      FROM artists b NATURAL JOIN released_by NATURAL JOIN releases
                      WHERE genre = 'Pop' )
ORDER BY name DESC;

 * postgresql://postgres:***@db:5432/postgres
35 rows affected.


name
Yello
WestBam
Underworld
U2
The Shamen
The Prodigy
The Human League
The Cure
The Chemical Brothers
The Beatles


## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

### Solution

1. Most queries consist of the following basic operations. They will reoccur throughout the whole semester. Watch out for them!
  * **Select**: select a subset of the rows/data records/items.
  * **Project**: select a subset of the properties/ attributes/columns.
  * **Join**: bring two datasets together based on a common attribute.
  * **Group**: divide the items/ rows/records into groups and summarize each group with a single value.
  * **Order**: order the items according to some criteria.

2. The usual syntax of a SQL query is `SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, OFFSET`. Ignoring the last 3 that only influence how the result is presented, `FROM` and `WHERE` happen before the grouping and `HAVING` and `SELECT` after.
  
3. We only describe *what* we want, not how this should be computed. We *declare* what our intent is. This shifts the implementation effort from the programmer to the database system. The hope is that the system has more information at hand, such as data size, data distribution, information about the hardware, in order to choose the best way to compute the result. This results into efficient computation with little effort from the programmer.
  
4. SQL is functional because results of a query can be used as input of another query, either in form of tables or in form of scalars. This makes SQL expressive.

5. If we denormalised the data and unify some of the tables, we no longer need to join them every time in order to get our results.

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [16]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X) AND c1.left_id != c2.right_id
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;

 * postgresql://postgres:***@db:5432/postgres
146 rows affected.


left_id,left_name,right_id,right_name,distance
29735,Coldplay,124541,Bon Jovi,1
29735,Coldplay,1654,DK,1
29735,Coldplay,392179,G Synth,1
29735,Coldplay,10916,Jan Johnston,1
29735,Coldplay,12236,Marco V,1
29735,Coldplay,4213,Max Graham,1
29735,Coldplay,92827,Michelle Nicol,1
29735,Coldplay,1279,Orbital,1
29735,Coldplay,239326,Walsh & Coutre,1
29735,Coldplay,10785,Angelo Badalamenti,2


### Solution
The query interprets the database as an undirected graph where *Artists* are nodes and edges exist between artists that have released an album together (i.e., collaborated). It then returns all of the artists which are at most `X` hops from a given artist in the graph. Even if the problem is simple when phrased with graph terminology, we can see that the query itself is quite complex: evidently, **SQL is not built for graph analysis!**. In the later weeks of the course, we will see tools that make graph analysis much more intuitive.

To understand why the query does what it does, think about how we can find all artists exactly `X` hops from a given artist. Firstly, we need to be able to easily retrieve all *directly* collaborating artists, i.e., the set of edges:

In [17]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists 
    JOIN released_by USING(artist_id)
    JOIN releases USING (release_id)
)
SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
        ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
FROM artist_releases AS ar1, artist_releases AS ar2
WHERE ar1.release_id = ar2.release_id
AND ar1.artist_id != ar2.artist_id
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


left_id,left_name,right_id,right_name,distance
1,The Persuader,89,Mr. James Barth,1
1,The Persuader,674,Stephan Grieder,1
3,Josh Wink,11,DJ Dozia,1
3,Josh Wink,789,Lil' Louis,1
3,Josh Wink,1268,Carl Cox,1
3,Josh Wink,2893,Woody McBride,1
3,Josh Wink,4367,DJ ESP,1
3,Josh Wink,11217,Size 9,1
3,Josh Wink,15346,Vinylgroover & The Red Hed,1
3,Josh Wink,27460,The Interpreters,1


where we first join artists with their releases under `artist_releases`, and then join `artist_releases` with itself to find all pairs of artists which have released the same `release_id`. We limit the query results to 10 for demonstration purposes.

We give this query the alias `collaborations` and join it with itself on a mutual `artist_id` to find all collaborations with two hops:

In [18]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists 
    JOIN released_by USING(artist_id)
    JOIN releases USING (release_id)
),
collaborations AS (
    SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
            ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
    FROM artist_releases AS ar1, artist_releases AS ar2
    WHERE ar1.release_id = ar2.release_id
    AND ar1.artist_id != ar2.artist_id
),
two_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
)
SELECT *
FROM two_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY right_name
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


left_id,left_name,right_id,right_name
29735,Coldplay,10785,Angelo Badalamenti
29735,Coldplay,41436,Benjamin Bates
29735,Coldplay,124541,Bon Jovi
29735,Coldplay,2737,BT
29735,Coldplay,29735,Coldplay
29735,Coldplay,184852,Cor Fijneman
29735,Coldplay,11101,Cosmic Gate
29735,Coldplay,981,Cylob
29735,Coldplay,1654,DK
29735,Coldplay,7090,Freefall


We can iteratively add indirections with another join like so:

In [19]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists 
    JOIN released_by USING(artist_id)
    JOIN releases USING (release_id)
),
collaborations AS (
    SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
            ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
    FROM artist_releases AS ar1, artist_releases AS ar2
    WHERE ar1.release_id = ar2.release_id
    AND ar1.artist_id != ar2.artist_id
),
two_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
),
three_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM two_hop_collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
),
four_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM three_hop_collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
)
SELECT *
FROM four_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY right_name
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


left_id,left_name,right_id,right_name
29735,Coldplay,36856,20 Fingers
29735,Coldplay,129491,2Freakz
29735,Coldplay,79578,50 Cent
29735,Coldplay,19195,About 2
29735,Coldplay,29862,Acid Ted
29735,Coldplay,15169,Adeva
29735,Coldplay,2684,Adonis
29735,Coldplay,165187,Africans With Mainframes
29735,Coldplay,6921,Age Of Love
29735,Coldplay,15959,Airfire


he problem with this approach is that the query *structure* depends on the number of hops you want to permit.

The query from the question, however, is independent of this number. You can just change `X` and leave the rest as it is. The trick is that it is a *recursive* SQL statement. Roughly speaking, a recursive statement consists of a base case statement connected via `UNION` (a concatenation of relations) with a statement the refers to the original statement. The recursive statement is executed recursively until it produces an empty result.

The query uses this mechanism to write `X_hop_collaborations` (repeated in isolation below): its base case are the direct `collaborations`. Each iteration joins the `collaborations` relation to find artists within one more hop for each artist already found. It keeps track of the distance from the original post by incrementing the value of `distance` by one in each iteration. When that attribute reaches the value of `X`, no records are produced by the recursive case, so recursion ends.

Consider: we have to join `collaborations` with an exponentially growing relation for each hop. In addition to the query being difficult to write, it becomes very inefficient as `X` increases.